<a href="https://colab.research.google.com/github/badonyi/adventofcode2024/blob/main/aov2024_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run this chunk before any of the daily challenges

In [ ]:
get_data <- function(x) sprintf('https://raw.githubusercontent.com/badonyi/adventofcode2024/refs/heads/main/data/%s.txt', x)

## Day 1

In [ ]:
input <- read.table(get_data('day_1'))

# part 1
sum(abs(sort(input$V1) - sort(input$V2)))

# part 2
sum(outer(input$V1, input$V2, `==`) * input$V1)

## Day 2

In [ ]:
input <- sapply(strsplit(readLines(get_data('day_2')), ' '), as.integer)

# part 1
sum(sapply(input, function(x) {
  d <- diff(x); all(abs(d) < 4) && (all(d < 0) || all(d > 0))
}))

# part 2
sum(sapply(input, function(x) {
  any(sapply(seq_along(x), function(i) {
    d <- diff(x[-i]); all(abs(d) < 4) && (all(d < 0) || all(d > 0))
  }))
}))


## Day 3

In [ ]:
input <- paste(readLines(get_data('day_3')), collapse = '')

mul_compute <- function(s) {
  eval(parse(text = paste(
    gsub(
      'mul\\((\\d+),(\\d+)\\)',
      '\\1*\\2',
      regmatches(s, gregexpr('mul\\((\\d+),(\\d+)\\)', s))[[1]]
    ),
    collapse = '+'
  )))
}

# part 1
mul_compute(input)

# part 2
donts <- strsplit(input, "don't\\(\\)")[[1]]
dos <- unlist(lapply(donts[-1], function(x) strsplit(x, 'do\\(\\)')[[1]][-1]))
mul_compute(paste(c(donts[1], dos), collapse = ''))

## Day 4

In [ ]:
mat <- do.call(rbind, strsplit(readLines(get_data('day_4')), NULL))
has_xmas <- function(x) paste(x, collapse = '') %in% c('XMAS', 'SAMX')

# part 1
n <- nrow(mat)
count <- 0
for (j in seq(n)) {
  for (i in seq(n)) {
    if (i <= n - 3) {
      count <- count + has_xmas(mat[j, i:(i + 3)])
      if (j < n - 2) count <- count + has_xmas(diag(mat[j:(j + 3), i:(i + 3)]))
      if (j > 3) count <- count + has_xmas(diag(mat[j:(j - 3), i:(i + 3)]))
    }
    if (j < n - 2) count <- count + has_xmas(mat[j:(j + 3), i])
  }
}
cat(count)

# part 2
sum(apply(
  X = expand.grid(seq(n), seq(n)),
  MARGIN = 1,
  FUN = function(g) {
    a <- g[1]
    b <- g[2]
    if (a > 1 && a < n && b > 1 && b <  n) {
      return(as.numeric(
        paste0(mat[a - 1, b - 1], mat[a - 1, b + 1],
               mat[a, b], mat[a + 1, b - 1],
               mat[a + 1, b + 1]) %in% c('SMASM', 'MSAMS', 'MMASS', 'SSAMM')))
    }
    return(0)
  }
))

## Day 5

In [ ]:
input <- readLines(get_data('day_5'))
sep <- which(input == '')
updates <- strsplit(input[(sep + 1):length(input)], ',')
pairs <- read.table(text = input[1:(sep - 1)], sep = '|')

is_correct <- function(update, pairs) {
  for (i in 1:nrow(pairs)) {
    if (pairs$V1[i] %in% update && pairs$V2[i] %in% update) {
      if (which(update == pairs$V1[i]) > which(update == pairs$V2[i])) {
        return(FALSE)
      }
    }
  }
  return(TRUE)
}

# part 1
sum(sapply(updates[sapply(updates, is_correct, pairs)],
           function(x) as.numeric(x[ceiling(length(x) / 2)])))

# part 2
correct_order <- function(update, pairs) {
  swap <- TRUE
  while (swap) {
    swap <- FALSE
    for (i in 1:nrow(pairs)) {
      if (pairs$V1[i] %in% update && pairs$V2[i] %in% update) {
        b_index <- which(update == pairs$V1[i])
        a_index <- which(update == pairs$V2[i])
        if (b_index > a_index) {
          update[c(a_index, b_index)] <- update[c(b_index, a_index)]
          swap <- TRUE
        }
      }
    }
  }
  return(update)
}

updates_wrong <- updates[!sapply(updates, is_correct, pairs)]
updates_right <- lapply(updates_wrong, correct_order, pairs)
sum(sapply(updates_right, function(x) as.numeric(x[ceiling(length(x) / 2)])))

## Day 6

In [ ]:
lab_map <- do.call(rbind, strsplit(readLines(get_data('day_6')), NULL))
nrows <- nrow(lab_map)
ncols <- ncol(lab_map)

directions <- c('up', 'right', 'down', 'left')
deltas <- list(up = c(-1L, 0L), right = c(0L, 1L), down = c(1L, 0L), left = c(0L, -1L))
dir_symbol <- c('^', '>', 'v', '<')

guard_start <- arrayInd(which(lab_map %in% dir_symbol), .dim = dim(lab_map))
guard_initial_pos <- c(guard_start[1], guard_start[2])
guard_initial_dir <- directions[match(lab_map[guard_initial_pos[1], guard_initial_pos[2]], dir_symbol)]

# part 1
visited <- matrix(FALSE, nrow = nrows, ncol = ncols)
visited[guard_initial_pos[1], guard_initial_pos[2]] <- TRUE

guard_pos <- guard_initial_pos
guard_dir <- guard_initial_dir
while (TRUE) {
  next_pos <- guard_pos + deltas[[guard_dir]]
  if (next_pos[1] < 1 || next_pos[1] > nrows || next_pos[2] < 1 || next_pos[2] > ncols) {
    break
  }
  if (lab_map[next_pos[1], next_pos[2]] == '#') {
    guard_dir <- directions[(match(guard_dir, directions) %% 4) + 1]
  } else {
    guard_pos <- next_pos
    visited[guard_pos[1], guard_pos[2]] <- TRUE
  }
}
cat(sum(visited), '\n')

# part 2
simulate_with_loop <- function(map) {
  guard_pos <- guard_initial_pos
  guard_dir <- guard_initial_dir

  visited_states <- new.env(hash = TRUE, parent = emptyenv(), size = 1000L)
  while (TRUE) {
    state <- (guard_pos[1] - 1L) * ncols + guard_pos[2] +
             (match(guard_dir, directions) - 1L) * nrows * ncols
    if (exists(as.character(state), envir = visited_states)) return(TRUE)
    assign(as.character(state), TRUE, envir = visited_states)

    next_pos <- guard_pos + deltas[[guard_dir]]
    if (any(next_pos < 1L) || next_pos[1] > nrows || next_pos[2] > ncols) return(FALSE)

    if (map[next_pos[1], next_pos[2]] == '#') {
      current_index <- match(guard_dir, directions)
      guard_dir <- directions[(current_index %% 4L) + 1L]
    } else {
      guard_pos <- next_pos
    }
  }
}

empty_spots <- which(lab_map == '.', arr.ind = TRUE)
total_spots <- nrow(empty_spots)
loop_count <- 0
pb <- txtProgressBar(min = 0, max = total_spots, style = 3)
on.exit(close(pb))
for (i in seq_len(total_spots)) {
  lab_map[empty_spots[i, 1], empty_spots[i, 2]] <- '#'
  if (simulate_with_loop(lab_map)) loop_count <- loop_count + 1
  lab_map[empty_spots[i, 1], empty_spots[i, 2]] <- '.'
  setTxtProgressBar(pb, i)
}
cat('\n', loop_count)